<a href="https://colab.research.google.com/github/aLehav/MLVideoDescriptionResearch/blob/main/BEATs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Link to BEATs Github](https://github.com/microsoft/unilm/tree/master/beats)

[Link to AudioSet](https://ieeexplore-ieee-org.libproxy2.usc.edu/stamp/stamp.jsp?tp=&arnumber=7952261)

In [1]:
!pip install boto3 --quiet
import io
import math
import torch
import torchaudio
import json
import os
import tarfile
import tempfile
import boto3
import matplotlib.pyplot as plt
import requests
from botocore import UNSIGNED
from botocore.config import Config
from IPython.display import Audio
!git clone https://github.com/microsoft/unilm.git --quiet
%cd unilm/beats
from Tokenizers import TokenizersConfig, Tokenizers
from BEATs import BEATs, BEATsConfig
%cd ../../
!git clone https://github.com/audioset/ontology.git --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 2.0 MB/s eta 0:00:00
/content/unilm/beats
/content


In [2]:
%cd ontology
f = open('ontology.json')
ontology = json.load(f)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd ../drive/MyDrive/video_summarization/
!ls

/content/ontology
Mounted at /content/drive
/content/drive/MyDrive/video_summarization
91IHQYk1IQM.mp3					 qqR6AEXwxoQ.mp3
91IHQYk1IQM.wav					 qqR6AEXwxoQ.wav
BEATs_iter3_plus_AS2M_finetuned_on_AS2M_cpt2.pt  Tokenizer_iter3_plus_AS2M.pt
BEATs_iter3_plus_AS2M.pt


In [9]:
# print(torchaudio.info("qqR6AEXwxoQ.wav"))
meta = torchaudio.info('91IHQYk1IQM.wav')
num_frames = meta.num_frames
waveform, sample_rate = torchaudio.load("91IHQYk1IQM.wav")
print(waveform.shape)

torch.Size([1, 1768432])


In [32]:
scaler = 2*sample_rate
i_range = range(math.ceil(num_frames/scaler))
def bot_slice(i):
  return i*scaler
def top_slice(i):
  return min((i+1)*scaler, num_frames)
waveforms = [waveform[:,bot_slice(i):top_slice(i)] for i in i_range]
# for wave in waveforms:
  # print(wave.shape)

In [34]:
audio_input_16khz = waveforms
inputs = len(waveforms)
length_per_input = scaler
# print(waveform.shape[1])
# len = audio_input_16khz.shape[1]
padding_mask = torch.zeros(inputs,length_per_input).bool()

In [30]:
print(len(waveforms))
print(waveforms[0].shape[1])

56
torch.Size([1, 32000])


In [24]:
# checkpoint = torch.load('Tokenizer_iter3_plus_AS2M.pt')
# checkpoint = torch.load('BEATs_iter3_plus_AS2M.pt')
checkpoint = torch.load('BEATs_iter3_plus_AS2M_finetuned_on_AS2M_cpt2.pt')

In [37]:
#for fine-tuned
cfg = BEATsConfig(checkpoint['cfg'])
BEATs_model = BEATs(cfg)
BEATs_model.load_state_dict(checkpoint['model'])
# BEATs_model.eval()  

<All keys matched successfully>

In [39]:
# probs = BEATs_model.extract_features(audio_input_16khz, padding_mask=padding_mask)[0]
def get_top_k_labels(audio_input_16khz, k):
  audio_length = audio_input_16khz.shape[1]
  audio_dim = audio_input_16khz.shape[0]
  padding_mask = torch.zeros(audio_dim,audio_length).bool()

  probs = BEATs_model.extract_features(audio_input_16khz, padding_mask=padding_mask)[0]
  for i, (top5_label_prob, top5_label_idx) in enumerate(zip(*probs.topk(k=k))):
      top5_label = [checkpoint['label_dict'][label_idx.item()] for label_idx in top5_label_idx]
      return (top5_label, top5_label_prob)
labels = [get_top_k_labels(wave, 5) for wave in waveforms]
print(labels)
  

[(['/m/04rlf', '/m/07y_7', '/m/04szw', '/m/0l14_3', '/m/0ggq0m'], tensor([0.8515, 0.7388, 0.4414, 0.2063, 0.0538], grad_fn=<UnbindBackward0>)), (['/m/04rlf', '/m/04szw', '/m/02p0sh1', '/m/07y_7', '/m/0ggq0m'], tensor([0.7774, 0.2165, 0.1589, 0.1263, 0.0920], grad_fn=<UnbindBackward0>)), (['/m/0192l', '/m/04rlf', '/m/04szw', '/m/085jw', '/m/02p0sh1'], tensor([0.6407, 0.6168, 0.2061, 0.1961, 0.0951], grad_fn=<UnbindBackward0>)), (['/m/04rlf', '/m/04szw', '/m/07y_7', '/m/02p0sh1', '/m/015vgc'], tensor([0.8715, 0.3749, 0.2871, 0.2507, 0.1501], grad_fn=<UnbindBackward0>)), (['/m/04rlf', '/m/01kcd', '/m/04szw', '/m/07gql', '/m/085jw'], tensor([0.6680, 0.3672, 0.3444, 0.2263, 0.1876], grad_fn=<UnbindBackward0>)), (['/m/04rlf', '/m/04szw', '/m/07y_7', '/m/02p0sh1', '/m/0l14_3'], tensor([0.8247, 0.6134, 0.5989, 0.2725, 0.1744], grad_fn=<UnbindBackward0>)), (['/m/04rlf', '/m/02p0sh1', '/m/0192l', '/m/04szw', '/m/0ggq0m'], tensor([0.8470, 0.5004, 0.3456, 0.2542, 0.1573], grad_fn=<UnbindBackward0>

In [40]:
import pandas as pd
df = pd.DataFrame(labels)

In [53]:
top_labels = [label[1][0][0] for label in labels.iterrows()]
print(top_labels)

['/m/04rlf', '/m/04rlf', '/m/0192l', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/09x0r', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/09x0r', '/m/09x0r', '/m/09x0r', '/m/07yv9', '/m/07yv9', '/m/04rlf', '/m/04rlf', '/m/04rlf', '/m/09x0r']


In [63]:
def ont_index (label):
  return next((i for i, item in enumerate(ontology) if item['id'] == label), -1)
english_labels = [ontology[ont_index(label)]['name'] for label in top_labels]
print(english_labels)

['Music', 'Music', 'Bagpipes', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Speech', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Music', 'Speech', 'Speech', 'Speech', 'Vehicle', 'Vehicle', 'Music', 'Music', 'Music', 'Speech']
